In [25]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [26]:
from ultralytics import YOLO
import csv
import os

Dataset: https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small?resource=download

In [44]:
# Create labels for train and val

# Create a dictionary to store image data
image_data = {}

# Define a dictionary to map class names to IDs
class_ids = {"monkey": 0, "cat": 1, "dog": 2}

def make_labels_txt(csv_file):

    # Create labels folder
    os.makedirs(f'data/{csv_file}/labels', exist_ok=True)

    # Open the CSV file
    with open(f'data/{csv_file}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)    
        next(reader)  # Skip the header row
        for row in reader:
            filename, img_width, img_height, class_name, xmin, ymin, xmax, ymax = row

            # Calculate the center, width and height of the bounding box
            x_center = (float(xmin) + float(xmax)) / 2 / float(img_width)
            y_center = (float(ymin) + float(ymax)) / 2 / float(img_height)
            width = (float(xmax) - float(xmin)) / float(img_width)
            height = (float(ymax) - float(ymin)) / float(img_height)

            # Get the class_id from the dictionary
            class_id = class_ids[class_name]

            # Add the data to the dictionary
            if filename in image_data:
                image_data[filename].append([class_id, x_center, y_center, width, height])
            else:
                image_data[filename] = [[class_id, x_center, y_center, width, height]]

    # Write the data to text files
    for filename, boxes in image_data.items():
        with open(os.path.join(f'data/{csv_file}/labels', f'{os.path.splitext(filename)[0]}.txt'), 'w') as file:
                for box in boxes:
                    file.write(' '.join(map(str, box)) + '\n')

In [45]:
# Make labels for train data
make_labels_txt("train")

# Make labels for val data
make_labels_txt("val")

In [47]:
# Open the file and print its contents
with open('data/train/labels/cats_and_monkeys_001.txt', 'r') as file:
    print(file.read())

1 0.5822784810126582 0.7894736842105263 0.5569620253164557 0.41353383458646614
0 0.7120253164556962 0.4116541353383459 0.5759493670886076 0.7556390977443609



In [48]:
# Open the file and print its contents
with open('data/val/labels/cats_and_monkeys_000.txt', 'r') as file:
    print(file.read())

1 0.540084388185654 0.6105341246290801 0.8818565400843882 0.6068249258160238
0 0.45569620253164556 0.5178041543026706 0.9071729957805907 0.913946587537092



In [42]:
import shutil

# Remove the 'labels' directories in 'train' and 'test'
# shutil.rmtree('data/train/labels')
shutil.rmtree('data/val/labels')

FileNotFoundError: [Errno 2] No such file or directory: 'data/val/labels'